

## Chapter 6: Real-Time Data - Subscriptions

So far, we have covered **Queries** (fetching data) and **Mutations** (modifying data). These operate on a "Request-Response" cycle: the client asks, and the server answers once. But modern applications often require real-time updates—think of chat apps, stock tickers, or live notifications.

GraphQL solves this with **Subscriptions**.

### 6.1 What are Subscriptions? WebSocket Fundamentals

A Subscription is a long-lived request where the server pushes data to the client whenever a specific event occurs.

**Pull vs. Push:**
*   **Queries (Pull):** "Give me the current list of messages."
*   **Subscriptions (Push):** "Tell me whenever a new message arrives."

**The Transport Layer:**
While Queries and Mutations typically use standard HTTP, Subscriptions usually run over **WebSockets**. A WebSocket connection stays open, allowing the server to send messages to the client at any time without the client having to request them.

**The Lifecycle:**
1.  **Handshake:** Client initiates a WebSocket connection with the server.
2.  **Subscription:** Client sends a subscription query over the socket.
3.  **Wait:** The connection stays open.
4.  **Push:** When an event happens on the server (e.g., a new row in the database), the server executes the subscription query and pushes the result through the open socket.
5.  **Unsubscribe:** The client sends a message to stop listening, or closes the connection.

### 6.2 Defining Subscription Schemas

Subscriptions are defined in the root `Subscription` type, similar to how Queries and Mutations are defined.

**Snippet:**

```graphql
type Subscription {
  "Subscribe to new messages in a specific chat room"
  messageSent(roomId: ID!): Message!
  
  "Subscribe to stock price changes"
  stockPriceUpdated(symbol: String!): Stock!
}

type Message {
  id: ID!
  text: String!
  author: User!
}

type Stock {
  symbol: String!
  price: Float!
}
```

**Explanation:**
Here, `messageSent` is a subscription field. It takes an argument (`roomId`) and returns a `Message` type. Unlike a query, this doesn't fetch the history; it waits for future events.

### 6.3 Subscription Arguments and Filters

In a real-world scenario, you rarely want to broadcast *every* event to *every* client. If I am in "Room A", I don't want to receive messages from "Room B".

While the schema looks simple, the logic happens in the **Resolver** or the **Subscription Manager**.

**Conceptual Resolver Logic (Server-Side):**

When a client subscribes, the server needs to know if a subsequent event is relevant to that client.

```javascript
// Pseudo-code for subscription resolver (e.g., using Apollo Server)
const resolvers = {
  Subscription: {
    messageSent: {
      // The "subscribe" method returns an AsyncIterator
      subscribe: (parent, { roomId }, { pubsub }) => {
        // We channel events specifically for this roomId
        return pubsub.asyncIterator([`MESSAGE_SENT_${roomId}`]);
      },
    },
  },
  
  Mutation: {
    sendMessage: (parent, { roomId, text }, { pubsub }) => {
      const newMessage = createMessage(text);
      
      // We publish the event to the specific channel
      pubsub.publish(`MESSAGE_SENT_${roomId}`, { messageSent: newMessage });
      
      return newMessage;
    }
  }
};
```

**Explanation:**
1.  **PubSub System:** Servers use a Publish-Subscribe (PubSub) system.
2.  **Dynamic Channels:** The resolver creates a dynamic channel name based on the argument (`MESSAGE_SENT_123`).
3.  **Filtering:** Only clients subscribed to `MESSAGE_SENT_123` receive the update.

**Client-Side Query:**

```graphql
subscription ListenToRoom {
  messageSent(roomId: "123") {
    text
    author {
      name
    }
  }
}
```

**Argument-based Filtering vs. Resolver Filtering:**
*   **Argument-based:** The schema forces the client to specify what they want (e.g., `roomId: ID!`). This is clean and explicit.
*   **Resolver Filtering:** For more complex logic (e.g., "Only notify me if the message mentions me"), you might use a `filter` function in the resolver logic to check the payload before sending it to the specific client.

### 6.4 Client-Side Subscription Management (Connection Lifecycle)

On the client side, managing a WebSocket connection requires handling connection state, reconnection logic, and subscription lifecycles.

**Using Apollo Client (React Example):**

Apollo Client provides a `useSubscription` hook that handles the complexities of re-subscribing if the network drops.

```javascript
import { useSubscription, gql } from '@apollo/client';

// Define the subscription
const MESSAGE_SUBSCRIPTION = gql`
  subscription OnMessageSent($roomId: ID!) {
    messageSent(roomId: $roomId) {
      text
    }
  }
`;

function ChatRoom({ roomId }) {
  // Hook automatically manages the connection
  const { data, loading } = useSubscription(
    MESSAGE_SUBSCRIPTION, 
    { variables: { roomId } }
  );

  return (
    <div>
      <h4>New Message:</h4>
      {loading ? <p>Waiting for messages...</p> : (
        <p>{data.messageSent.text}</p>
      )}
    </div>
  );
}
```

**Key Behaviors:**
1.  **Mount:** When the component mounts, the hook opens the subscription.
2.  **Update:** When the server pushes data, the `data` variable updates, triggering a re-render.
3.  **Unmount:** When the component unmounts, the hook automatically unsubscribes, saving server resources.

**Handling Connection Drops:**
In production, WebSockets can disconnect (bad network). Robust GraphQL clients use a `WebSocketLink` with retry logic. When the connection is restored, the client typically re-sends all active subscriptions to ensure no events were missed.

---

### 🚀 Next Up: Chapter 7 - Building a GraphQL Server

**Summary:** We have learned the entire language of GraphQL: Queries, Mutations, and Subscriptions. Now it is time to build the engine that powers them. In Chapter 7, we will dive into server-side implementation. We will set up a Node.js server, learn the anatomy of a Resolver, connect to databases, and structure our project for scalability. This chapter moves us from theory to practical application.